## Load Modules

In [1]:
# Signal Preprocessing Methods
from preprocessModule import *

# Transform preprocessed signal into an input formation
from transformModule import *

# Plot methods for various input data
from plotModule import *

# 1. DEAP Dataset

### DEAP Dataset Description
* __32 participants__
* __40 clips__ for each participant
* Evaluation : __Valence, Arousal, Dominance, Like__ (1 to 9)

### Setting & Initialization

In [2]:
DEAP_channels = {"FP1":0, "AF3":1, "F3":2, "F7":3,
                 "FC5":4, "FC1":5, "C3":6, "T7":7,
                 "CP5":8, "CP1":9, "P3":10,
                 "P7":11, "PO3":12, "O1":13, 
                 "OZ":14, "PZ":15, "FP2":16, 
                 "AF4":17, "FZ":18, "F4":19,
                 "F8":20, "FC6":21, "FC2":22,
                 "CZ":23, "C4":24, "T8":25,
                 "CP6":26, "CP2":27, "P4":28,
                 "P8":29, "PO4":30, "O2":31}

DEAP_all_channel_names = list(DEAP_channels.keys())
DEAP_all_channel_values = list(DEAP_channels.values())

In [3]:
# Initialization ============================
# Label list
label_list = [] # list of tuple (V,A,D,L)

# Input list 
fftMap_list = [] # Fast Fourier Transform Map
base_fftMap_list = [] # base(3sec) fft map list

fd_list = [] # FD pattern
base_fd_list = [] 
# ===========================================

# Parameters ===============================
freqs = [freq for freq in range(4,46,1)]
chosen_channels = DEAP_all_channel_values
# openBCI default channels
#chosen_channels = [0, 16, 6, 24, 11, 20, 13, 31] # ["FP1", "FP2", "C3", "C4", "P7", "P8", "O1", "O2"]

sf = 128 
chunk_size = 10 * sf
overlap = 5 * sf
# ==========================================

### Traverse Dataset

In [313]:
import time
import pickle

eeg_path = "EEG dataset/DEAP/physiological_recordings/data_preprocessed_python/data_preprocessed_python/"

# [ Loop 1 ] 32 Participants
for participant_id in range(1, 33):
    file_name = "s" + str(int(participant_id/10)) + str(participant_id%10) + ".dat"
    s = pickle.load(open(eeg_path + file_name, 'rb'), encoding='latin1')
    
    labels = s['labels']
    data = s['data']
    
    # [ Loop 2 ] 40 Trials
    for trial_id in range(0, 40):      
        # label 
        V, A, D, L = labels[trial_id]
        
        base_signal = data[trial_id][:, :sf * 3] # 3sec
        total_signal = data[trial_id][:, sf * 3:] # 60sec
        
        # all_chunks = [[start//sf, (start+chunk_size)//sf] for start in range(0, sf*60-chunk_size+1, chunk_size-overlap)]
        # print("Total %d chunks",%(len(all_chunks))); print(all_chunks)
        
        all_chunks = [total_signal[:, start:start+chunk_size] 
                     for start in range(0, sf*60-chunk_size+1, chunk_size-overlap)]
        
        # processing base signal==================================
        base_fftMap = computefftMap(base_signal, chosen_channels, freqs, sf)
        base_fftMap_list.append(base_fftMap)
        base_fd = computeFD(base_signal, chosen_channels)
        base_fd_list.append(base_fd)
        # ========================================================
        
        # [ Loop 3 ] Sliding window
        for chunk in all_chunks:
            # fftMap (2d array) ======================
            fftMap = computefftMap(chunk, chosen_channels, freqs, sf)    
            fftMap_list.append(fftMap)
            # ========================================
            # FD pattern (1d array)====================
            fd = computeFD(chunk, chosen_channels)
            fd_list.append(fd)
            # =========================================
            label_list.append((V,A,D,L))
    print("Participant %d Ended"%(participant_id))
    print(time.strftime('%c', time.localtime(time.time())))

1 분석 끝
Mon Apr 13 02:51:22 2020
2 분석 끝
Mon Apr 13 02:51:29 2020
3 분석 끝
Mon Apr 13 02:51:37 2020
4 분석 끝
Mon Apr 13 02:51:43 2020
5 분석 끝
Mon Apr 13 02:51:50 2020
6 분석 끝
Mon Apr 13 02:51:57 2020
7 분석 끝
Mon Apr 13 02:52:03 2020
8 분석 끝
Mon Apr 13 02:52:10 2020
9 분석 끝
Mon Apr 13 02:52:18 2020
10 분석 끝
Mon Apr 13 02:52:27 2020
11 분석 끝
Mon Apr 13 02:52:35 2020
12 분석 끝
Mon Apr 13 02:52:42 2020
13 분석 끝
Mon Apr 13 02:52:50 2020
14 분석 끝
Mon Apr 13 02:52:58 2020
15 분석 끝
Mon Apr 13 02:53:05 2020
16 분석 끝
Mon Apr 13 02:53:13 2020
17 분석 끝
Mon Apr 13 02:53:21 2020
18 분석 끝
Mon Apr 13 02:53:28 2020
19 분석 끝
Mon Apr 13 02:53:35 2020
20 분석 끝
Mon Apr 13 02:53:42 2020
21 분석 끝
Mon Apr 13 02:53:49 2020
22 분석 끝
Mon Apr 13 02:53:57 2020
23 분석 끝
Mon Apr 13 02:54:04 2020
24 분석 끝
Mon Apr 13 02:54:11 2020
25 분석 끝
Mon Apr 13 02:54:19 2020
26 분석 끝
Mon Apr 13 02:54:26 2020
27 분석 끝
Mon Apr 13 02:54:33 2020
28 분석 끝
Mon Apr 13 02:54:40 2020
29 분석 끝
Mon Apr 13 02:54:48 2020
30 분석 끝
Mon Apr 13 02:54:56 2020
31 분석 끝
Mon Apr 13 

In [364]:
noc = len(fftMap_list) // 1280 # num of chunks
len(label_list) == len(fftMap_list) # True

True

### Save & Load

In [266]:
def get_data_index(p_id, t_id, c_id, noc):
    return (noc * 40) * p_id + noc * t_id + c_id

def get_base_index(p_id, t_id):
    return 40 * p_id + t_id

# get p_id, t_id, c_id
def from_data_index(index, noc):
    # return index//40, index%40
    p_id,remain = index // (noc*40), index % (noc*40)
    return p_id, remain//noc, remain%noc

#### Save

In [337]:
data_list = []
save_option = "origin" # "diff"

for i in range(len(fftMap_list)):
    # activated signals (60sec)
    if save_option == "origin":
        data_list.append([fftMap_list[i], label_list[i]]) # **
        
    # 60sec(activate) - 3sec(base)
    elif save_option == "diff":
        p_id, t_id, c_id = from_data_index(i)
        base_index = get_base_index(p_id, t_id)
        
        diff = np.array(scale(fd_list[i])) - np.array(scale(base_fd_list[base_index]))
        data_list.append([diff, label_list[i]])

In [338]:
with open('DEAP_fftMap_10sec_5over.pickle', 'wb') as f:
    pickle.dump(data_list, f, pickle.HIGHEST_PROTOCOL)